In [7]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.model_selection import train_test_split
import zipfile
import os
import numpy as np
import matplotlib.pyplot as plt
import joblib

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.model_selection import train_test_split
import zipfile
import os
import numpy as np
import matplotlib.pyplot as plt
import joblib

ModuleNotFoundError: No module named 'tensorflow'

In [11]:
# Extract dataset
zip_file_path = 'assets/rockpaperscissors.zip'
extract_folder = 'assets/extracted/'

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_folder)

In [12]:
# Define paths
base_dir = 'assets/extracted/rockpaperscissors/rps-cv-images'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

In [ ]:
# Split the dataset into train and validation sets
data_gen = ImageDataGenerator(
    rescale=1./255, 
    validation_split=0.4,
    rotation_range=20, 
    width_shift_range=0.2, 
    height_shift_range=0.2,
    shear_range=0.2, 
    zoom_range=0.2, 
    horizontal_flip=True, 
    fill_mode='nearest')

datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.3,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_generator = data_gen.flow_from_directory(
    base_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='training')

validation_generator = data_gen.flow_from_directory(
    base_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='validation')

In [ ]:
# Build the model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(3, activation='softmax')
])

In [15]:
# Compile the model
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
# Train the model
history = model.fit(train_generator,
                    steps_per_epoch=train_generator.samples // 32,
                    validation_steps=validation_generator.samples // 32,
                    epochs=30,
                    validation_data=validation_generator)

In [ ]:
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend()
plt.show()

In [ ]:
# Access the values from the history object
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

# Print the values
print("Training Accuracy: ", acc[-1])
print("Validation Accuracy: ", val_acc[-1])

In [ ]:
# Save model
joblib.dump(model, 'model.keras')

In [ ]:
def predict_image(image_path, model, class_names):
    # Load and preprocess image
    img = image.load_img(image_path, target_size=(150, 150))
    img_tensor = image.img_to_array(img)
    img_tensor = np.expand_dims(img_tensor, axis=0)
    img_tensor /= 255.  # Normalization

    # Prediction
    prediction = model.predict(img_tensor)
    predicted_class_index = np.argmax(prediction[0])
    predicted_class = class_names[predicted_class_index]
    
    # Show result
    plt.imshow(img)
    plt.title(f"Predicted: {predicted_class}")
    plt.axis("off")
    plt.show()
    
    return predicted_class

# List class from train_generator
class_names = list(train_generator.class_indices.keys()) 

# Input image
try:
    from google.colab import files
    uploaded = files.upload()
    image_paths = list(uploaded.keys()) 
except ImportError:
    image_paths = [input("Masukkan path gambar: ")]

# Uploaded image prediction
for image_path in image_paths:
    predicted_class = predict_image(image_path, model, class_names)
